### Initialization

In [0]:
# Download spaCy models
!python -m spacy download en_core_web_md

import json

import en_core_web_md
import pandas as pd
from IPython.display import display

# pandas display settings
pd.set_option('display.max_columns', 10)
pd.set_option('max_colwidth', 1000)
pd.set_option('display.width', 1000)

# Initialize spaCy pipeline
SPACY = en_core_web_md.load()

RANDOM_SEED = 42  # for reproducibility

# Load Yelp reviews
reviews = []
with open("data/reviewCleaned.json", 'r', encoding='latin-1') as f:
    for line in f:
        reviews.append(json.loads(line))

column_order = ['business_id', 'text', 'stars']
YELP_REVIEWS = pd.DataFrame.from_records(reviews, columns=column_order)
YELP_REVIEWS.infer_objects()

print(f"\nFinished loading {len(YELP_REVIEWS)} to pandas DataFrame.")
print("\nSample records:")

display(YELP_REVIEWS.head())

print("\nPreliminary analysis:")
YELP_REVIEWS.describe()

# Global variables (reusable in other code cells)
# 1. SPACY: spaCy model for linguistic analysis
# 2. RANDOM_SEED: random seed for random generation
# 3. YELP_REVIEWS: pandas DataFrame containing Yelp reviews

     |████████████████████████████████| 95.4MB 1.5MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126237 sha256=22ff49e8a5bb6fa618a0311bda510e12d5c0bb5af977521762eb4cf06a92d2ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-2t2nu24i/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')

Finished loading 15222 to pandas DataFrame.

Sample records:


,business_id,text,stars
0,ZBE-H_aUlicix_9vUGQPIQ,"We had my Mother's Birthday Party here on 10/29/16. What a Great time we all had. The food, music and waiters were Great!!! Thanks Lyles!!!",5.0
1,e-YnECeZNt8ngm0tu4X9mQ,"Good Korean grill near Eaton Centre. The marinate is good. We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly. The fish fillet was bland and liver was meh. Salmon and chicken was really flavourable. Such a fun place to eat at for a date or group of friends. Even alone. No judgments here. \nThe staff is attentive, nice and considerate. Bigger groups will most likely be seated on the second floor which is way bigger.\nCaution: will smell like BBQ grill after.",4.0
2,j7HO1YeMQGYo3KibMXZ5vg,"Was recommended to try this place by few people and today was my first time here. All I can say is, I am coming back very soon.\n\nSERVICE\nWasn't sure if the guy was the owner but he was friendly and talked story while we waited for our food. Loved it!! Food came out within 10 min. \n\nFOOD\nTried hamburger steak and it was so delicious. Gravy/sauce they put on the hamburger steak was perfect! Also came with onion rings on top which I love. Chicken katsu was amazing! Chicken katsu here is crunchy and surprisingly has a flavor by itself that you really don't need a sauce for it. Best chicken katsu I had. \n\nOVERALL\nIt was a journey to get to this place as it took about 30min from my house but the service and food here made it worth the drive. I also love how they had a poster of Keali'i Reichel. (They had other posters but Keali'i Reichel happens to be my favorite). Place is clean, service is fast and friendly and food is delicious. What more could you ask for?",5.0
3,7e3PZzUpG5FYOTGt3O3ePA,"Ambience: Would not expect something this nice at Cannery Hotel but it is the nicest looking restaurant there. More for couples than group gatherings.\n\nService: The ambience & food make up for this, which unfortunately for us, the service has been terrible. We have come fairly close to restaurant closing both times (within the hour), but they do close very early for Vegas. The staff makes it VERY clear that they want to go home right from the start in hurrying orders and are more aggressive as time goes on. Unfortunate.\n\nFood: Very good. A little salty on some items during our first visit but good overall and again, warrants the overall 3 stars. Steak. Scallops wrapped in bacon. Calamari. Cobb salad. etc.",3.0
4,vuHzLZ7nAeT-EiecOkS5Og,"Absolutely the WORST pool company that I have EVER had to deal with. The customer service is horrible. After leaving many messages over the course of a few weeks I was only able to contact them when I called them AGAIN. I asked to speak with the actual pool tech who initially came to my house. The RUDE lady on the phone told me that she was more than capable to answer my questions - about a pump that SHE HAS NOT SEEN, and about a conversation I had with the tech THAT SHE DID NOT HEAR. \n\nI was assigned to them by my home warranty company, and I will be filing a serious complaint with them and the BBB. I was told to take the cash out option from the warranty company for the part and then they would do the work and I could just pay them directly. After I received the cash out and called to schedule the appointment I was told that I need to replace the entire pool pump system and that would cost an additional $400 and that there was an electrical problem and that it would cost...",1.0



Preliminary analysis:


,stars
count,15222.000000
mean,3.646367
std,1.455229
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


### Segment review into individual sentence


In [0]:
from collections import Counter

from tqdm import tqdm

YELP_REVIEWS['segments'] = ""
for index, row in tqdm(list(YELP_REVIEWS.iterrows())):
    doc = SPACY(row['text'])
    YELP_REVIEWS.at[index, 'segments'] = [seg.text.strip() for seg in doc.sents]

display(YELP_REVIEWS[['segments', 'stars']])

100%|██████████| 15222/15222 [06:45<00:00, 32.08it/s]


,segments,stars
0,"[We had my Mother's Birthday Party here on 10/29/16., What a Great time we all had., The food, music and waiters were Great!!!, Thanks Lyles!!!]",5.0
1,"[Good Korean grill near Eaton Centre., The marinate is good., We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly., The fish fillet was bland and liver was meh., Salmon and chicken was really flavourable., Such a fun place to eat at for a date or group of friends., Even alone., No judgments here., The staff is attentive, nice and considerate., Bigger groups will most likely be seated on the second floor which is way bigger., Caution: will smell like BBQ grill after.]",4.0
2,"[Was recommended to try this place by few people and today was my first time here., All I can say is, I am coming back very soon., SERVICE\nWasn't sure if the guy was the owner but he was friendly and talked story while we waited for our food., Loved it!!, Food came out within 10 min., FOOD\nTried hamburger steak and it was so delicious., Gravy/sauce, they put on the hamburger steak, was perfect!, Also came with onion rings on top which I love., Chicken katsu was amazing!, Chicken katsu here is crunchy and surprisingly has a flavor by itself that you really don't need a sauce for it., Best chicken katsu I had., OVERALL, It was a journey to get to this place as it took about 30min from my house but the service and food here made it worth the drive., I also love how they had a poster of Keali'i Reichel., (They had other posters but Keali'i Reichel happens to be my favorite)., Place is clean, service is fast and friendly and food is delicious., What more could you ask for?]",5.0
3,"[Ambience: Would not expect something this nice at Cannery Hotel, but it is the nicest looking restaurant there., More for couples than group gatherings., Service: The ambience & food make up for this, which unfortunately for us, the service has been terrible., We have come fairly close to restaurant closing both times (within the hour), but they do close very early for Vegas., The staff makes it VERY clear that they want to go home right from the start in hurrying orders and are more aggressive as time goes on., Unfortunate., Food:, Very good., A little salty on some items during our first visit but, good overall and again, warrants the overall 3 stars., Steak., Scallops wrapped in bacon., Calamari., Cobb salad., etc.]",3.0
4,"[Absolutely the WORST pool company that I have EVER had to deal with., The customer service is horrible., After leaving many messages over the course of a few weeks I was only able to contact them when I called them AGAIN., I asked to speak with the actual pool tech who initially came to my house., The RUDE lady on the phone told me that she was more than capable to answer my questions - about a pump that SHE HAS NOT SEEN, and about a conversation I had with the tech THAT SHE DID NOT HEAR., I was assigned to them by my home warranty company, and I will be filing a serious complaint with them and the BBB., I was told to take the cash out option from the warranty company for the part, and then they would do the work, and I could just pay them directly., After I received the cash out and called to schedule the appointment I was told that I need to replace the entire pool pump system and that would cost an additional $400 and that there was an electrical problem and that it would cost ...",1.0
...,...,...
15217,"[This was the worst experience ever., So much so that I'm in the parking lot writing this., Girl took off my entire eyebrow basically when I asked her to clean them up ONLY., Not to mention I'm taking maternity photos tomorrow., I literally cried in the chair., Never again., HORRIBLE]",1.0
15218,"[We come here every time we hit Vegas!, A giant thank you to The Pub bar staff for always making us feel like it's our birthday!, The food is great and the service is the best., If you haven't been - stop in!, It will be your home away from home.]",5.0
15219,"[As locals we us

### Extract noun-adjective pair from each review segment

##### Approach 1: using dependency parsing

In [0]:
from tqdm import tqdm

YELP_REVIEWS['noun-adjective'] = ""
for index, row in tqdm(list(YELP_REVIEWS.iterrows())):
    if row['segments'] != "":
        noun_adjectives = set()
        for segment in row['segments']:
            doc = SPACY(segment)
            for token in doc:
                noun = None
                adjective = None
                if token.pos_ == 'ADJ' and token.head.pos_ == 'NOUN':
                    noun = token.head.text
                    adjective = token.text
                elif token.pos_ == 'NOUN' and token.head.pos_ == 'ADJ':
                    noun = token.text
                    adjective = token.head.text
                elif token.pos_ == 'ADJ' and token.head.pos_ == 'VERB':
                    for t in list(token.head.lefts)[::-1]:
                        if t.pos_ == 'NOUN':
                            noun = t.text
                            break
                    adjective = token.text

                if noun is not None and adjective is not None:
                    noun_adjectives.add((noun, adjective))

        YELP_REVIEWS.at[index, 'noun-adjective'] = list(noun_adjectives)

num_empty = (YELP_REVIEWS['noun-adjective'].str.len() == 0).sum()
print(f"\nNumber of reviews without noun-adjective pairs: {num_empty} ({num_empty / len(YELP_REVIEWS) * 100}%)")
display(YELP_REVIEWS[['text', 'stars', 'noun-adjective']][:50])

100%|██████████| 15222/15222 [25:37<00:00,  9.90it/s]


Number of reviews without noun-adjective pairs: 308 (2.0233872027328865%)


,text,stars,noun-adjective
0,"We had my Mother's Birthday Party here on 10/29/16. What a Great time we all had. The food, music and waiters were Great!!! Thanks Lyles!!!",5.0,"[(time, Great), (food, Great)]"
1,"Good Korean grill near Eaton Centre. The marinate is good. We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly. The fish fillet was bland and liver was meh. Salmon and chicken was really flavourable. Such a fun place to eat at for a date or group of friends. Even alone. No judgments here. \nThe staff is attentive, nice and considerate. Bigger groups will most likely be seated on the second floor which is way bigger.\nCaution: will smell like BBQ grill after.",4.0,"[(grill, Good), (grill, Korean), (floor, second), (place, fun), (liver, meh), (staff, attentive), (groups, Bigger), (marinate, good), (fillet, bland)]"
2,"Was recommended to try this place by few people and today was my first time here. All I can say is, I am coming back very soon.\n\nSERVICE\nWasn't sure if the guy was the owner but he was friendly and talked story while we waited for our food. Loved it!! Food came out within 10 min. \n\nFOOD\nTried hamburger steak and it was so delicious. Gravy/sauce they put on the hamburger steak was perfect! Also came with onion rings on top which I love. Chicken katsu was amazing! Chicken katsu here is crunchy and surprisingly has a flavor by itself that you really don't need a sauce for it. Best chicken katsu I had. \n\nOVERALL\nIt was a journey to get to this place as it took about 30min from my house but the service and food here made it worth the drive. I also love how they had a poster of Keali'i Reichel. (They had other posters but Keali'i Reichel happens to be my favorite). Place is clean, service is fast and friendly and food is delicious. What more could you ask for?",5.0,"[(posters, other), (katsu, amazing), (service, fast), (drive, worth), (time, first), (food, delicious), (service, worth), (Place, clean), (katsu, crunchy), (katsu, Best), (story, friendly), (people, few)]"
3,"Ambience: Would not expect something this nice at Cannery Hotel but it is the nicest looking restaurant there. More for couples than group gatherings.\n\nService: The ambience & food make up for this, which unfortunately for us, the service has been terrible. We have come fairly close to restaurant closing both times (within the hour), but they do close very early for Vegas. The staff makes it VERY clear that they want to go home right from the start in hurrying orders and are more aggressive as time goes on. Unfortunate.\n\nFood: Very good. A little salty on some items during our first visit but good overall and again, warrants the overall 3 stars. Steak. Scallops wrapped in bacon. Calamari. Cobb salad. etc.",3.0,"[(something, nice), (restaurant, nicest), (visit, first), (staff, clear), (stars, overall), (service, terrible)]"
4,"Absolutely the WORST pool company that I have EVER had to deal with. The customer service is horrible. After leaving many messages over the course of a few weeks I was only able to contact them when I called them AGAIN. I asked to speak with the actual pool tech who initially came to my house. The RUDE lady on the phone told me that she was more than capable to answer my questions - about a pump that SHE HAS NOT SEEN, and about a conversation I had with the tech THAT SHE DID NOT HEAR. \n\nI was assigned to them by my home warranty company, and I will be filing a serious complaint with them and the BBB. I was told to take the cash out option from the warranty company for the part and then they would do the work and I could just pay them directly. After I received the cash out and called to schedule the appointment I was told that I need to replace the entire pool pump system and that would cost an additional $400 and that there was an electrical problem and that it would cost...",1.0,"[(service, horrible), (weeks, few), (system, entire), (problem, electrical), (money, additional), (messages, many

In [0]:
# display results without noun-adjective pair
empty = YELP_REVIEWS[YELP_REVIEWS['noun-adjective'].str.len() == 0]
display(empty[['segments']])

,segments
5,"[I love love their Kalbi, I always order it, the sauce is what makes it really good, ..hmm think I wanna eat that today.i didn't like their noodles to sweet..]"
52,"[My wife and I looked forward to eating here for weeks., We decided to get one of our favorites tonight, bbq pork ribs., Let me say... extremely disapointed!, Neither one of us would recommend this place.]"
167,"[Keep trying to unsubscribe from their mailing list, though today they sent me 7 SPAM emails., Stop it!, They also have a link to review their spa's on Yelp on the e-mail - though it just goes to the Yelp homepage., They obviously don't care much about their customers., It was so long ago, I was there, I can't remember the service.]"
248,"[Of all the restaurants in the courtyard, this is undoubtedly the least impressive., I wouldn't do it if I were you.....]"
256,"[Service and quality of food have declined., This Outback used to be very good., Now is a waste of money.]"
...,...
15077,"[Find another property company, they are neglegant in doing walk thrus, following thru with tenants etc., They allowed tenants to thrash our property., We were clients for 3 years and they never did their job, followed up or returned calls on time, when we decided to end our contract they did not communicate nor explain the process, and they tried to force us to use their maintenance company who did not do their job, even after we paid for it., Save your money and go somewhere else where they value their clients.]"
15140,"[Quick, reliable, and very knowledgeable., Jose and lucil were awesome, I recommend this play to everyone!!]"
15145,"[The waiter kept wiping his nose with his hands and serving people their food., Without washing his hands., Ugh., Unfortunately I didn't catch his name., His hat was turned backwards.]"
15154,"[Go see Marieja (so sorry if I spelled that wrong)!, She is fantastic., My mom & I both visited her for spur-of-the-moment eyebrow waxes and tints and our eyebrows have seriously never looked better., She is totally professional and really knows what she's doing., So happy I found her., I highly recommend her, and I'm very picky about who does my eyebrows.]"


In [0]:
import random
from collections import Counter

random.seed(RANDOM_SEED)

def get_top_n_pairs(group, n):
    counter = Counter()
    for pairs in group['noun-adjective']:
        for pair in pairs:
            counter[(pair[0].lower(), pair[1].lower())] += 1
    
    return counter.most_common(n)

for business_id, group in random.sample(list(YELP_REVIEWS.groupby('business_id')), 5):
    print(f"Top 10 noun-adjective pair for business '{business_id}':")
    top_pairs = get_top_n_pairs(group, 10)
    for i, (pair, count) in enumerate(top_pairs):
        print(f"{i + 1:>3}. {'-'.join(pair):<25} {count}")
    
    print("=" * 180)

Top 10 noun-adjective pair for business 'Gr_TkW3iFdgahixONGBsww':
  1. desk-front                12
  2. staff-friendly            9
  3. hotel-clean               6
  4. hotel-great               6
  5. hotel-best                5
  6. food-good                 5
  7. hotel-beautiful           5
  8. hotel-nice                5
  9. hotels-other              4
 10. location-great            4
Top 10 noun-adjective pair for business '2xrpo-LXV9uGIwpvy0dwUw':
  1. car-clean                 10
  2. job-great                 8
  3. time-first                7
  4. staff-friendly            7
  5. job-good                  6
  6. locations-other           5
  7. wash-basic                4
  8. wash-best                 4
  9. time-next                 4
 10. wash-free                 4
Top 10 noun-adjective pair for business 'WO3L0pmtAO8ozspmaVdHIg':
  1. food-chinese              25
  2. food-fast                 11
  3. night-late                10
  4. dog-hot                   8
  5. 

##### Approach 2: using exhaustive combinations of all noun-adjective pair

In [0]:
from itertools import product

from tqdm import tqdm

YELP_REVIEWS['noun-adjective'] = ""
for index, row in tqdm(list(YELP_REVIEWS.iterrows())):
    if row['segments'] != "":
        nouns = []
        adjectives = []
        for segment in row['segments']:
            doc = SPACY(segment)
            for token in doc:
                if token.pos_ == 'ADJ':
                    adjectives.append(token.text)
                elif token.pos_ == 'NOUN':
                    nouns.append(token.text)
        
        pairs = list(product(nouns, adjectives))
        YELP_REVIEWS.at[index, 'noun-adjective'] = pairs

num_empty = (YELP_REVIEWS['noun-adjective'].str.len() == 0).sum()
print(f"\nNumber of reviews without noun-adjective pairs: {num_empty} ({num_empty / len(YELP_REVIEWS) * 100}%)")
display(YELP_REVIEWS[['text', 'stars', 'noun-adjective']])

100%|██████████| 15222/15222 [25:58<00:00,  8.10it/s]



Number of reviews without noun-adjective pairs: 73 (0.47956904480357376%)


,text,stars,noun-adjective
0,"We had my Mother's Birthday Party here on 10/29/16. What a Great time we all had. The food, music and waiters were Great!!! Thanks Lyles!!!",5.0,"[(time, Great), (time, Great), (food, Great), (food, Great), (music, Great), (music, Great), (waiters, Great), (waiters, Great)]"
1,"Good Korean grill near Eaton Centre. The marinate is good. We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly. The fish fillet was bland and liver was meh. Salmon and chicken was really flavourable. Such a fun place to eat at for a date or group of friends. Even alone. No judgments here. \nThe staff is attentive, nice and considerate. Bigger groups will most likely be seated on the second floor which is way bigger.\nCaution: will smell like BBQ grill after.",4.0,"[(grill, Good), (grill, Korean), (grill, good), (grill, bland), (grill, meh), (grill, flavourable), (grill, fun), (grill, attentive), (grill, nice), (grill, considerate), (grill, Bigger), (grill, second), (grill, bigger), (marinate, Good), (marinate, Korean), (marinate, good), (marinate, bland), (marinate, meh), (marinate, flavourable), (marinate, fun), (marinate, attentive), (marinate, nice), (marinate, considerate), (marinate, Bigger), (marinate, second), (marinate, bigger), (beef, Good), (beef, Korean), (beef, good), (beef, bland), (beef, meh), (beef, flavourable), (beef, fun), (beef, attentive), (beef, nice), (beef, considerate), (beef, Bigger), (beef, second), (beef, bigger), (ox, Good), (ox, Korean), (ox, good), (ox, bland), (ox, meh), (ox, flavourable), (ox, fun), (ox, attentive), (ox, nice), (ox, considerate), (ox, Bigger), (ox, second), (ox, bigger), (liver, Good), (liver, Korean), (liver, good), (liver, bland), (liver, meh), (liver, flavourable), (liver, fun), (liver, att..."
2,"Was recommended to try this place by few people and today was my first time here. All I can say is, I am coming back very soon.\n\nSERVICE\nWasn't sure if the guy was the owner but he was friendly and talked story while we waited for our food. Loved it!! Food came out within 10 min. \n\nFOOD\nTried hamburger steak and it was so delicious. Gravy/sauce they put on the hamburger steak was perfect! Also came with onion rings on top which I love. Chicken katsu was amazing! Chicken katsu here is crunchy and surprisingly has a flavor by itself that you really don't need a sauce for it. Best chicken katsu I had. \n\nOVERALL\nIt was a journey to get to this place as it took about 30min from my house but the service and food here made it worth the drive. I also love how they had a poster of Keali'i Reichel. (They had other posters but Keali'i Reichel happens to be my favorite). Place is clean, service is fast and friendly and food is delicious. What more could you ask for?",5.0,"[(place, few), (place, first), (place, sure), (place, friendly), (place, delicious), (place, perfect), (place, amazing), (place, crunchy), (place, Best), (place, worth), (place, other), (place, favorite), (place, clean), (place, fast), (place, friendly), (place, delicious), (place, more), (people, few), (people, first), (people, sure), (people, friendly), (people, delicious), (people, perfect), (people, amazing), (people, crunchy), (people, Best), (people, worth), (people, other), (people, favorite), (people, clean), (people, fast), (people, friendly), (people, delicious), (people, more), (today, few), (today, first), (today, sure), (today, friendly), (today, delicious), (today, perfect), (today, amazing), (today, crunchy), (today, Best), (today, worth), (today, other), (today, favorite), (today, clean), (today, fast), (today, friendly), (today, delicious), (today, more), (time, few), (time, first), (time, sure), (time, friendly), (time, delicious), (time, perfect), (time, amazing)..."
3,"Ambience: Would not expect something this nice at Cannery Hotel but it is the nicest looking restaurant there. More for couples than group gatherings.\n\nService: The ambience & food make up for this, which unfor

In [0]:
# display results without noun-adjective pair
empty = YELP_REVIEWS[YELP_REVIEWS['noun-adjective'].str.len() == 0]
display(empty[['segments']])

,segments
52,"[My wife and I looked forward to eating here for weeks., We decided to get one of our favorites tonight, bbq pork ribs., Let me say... extremely disapointed!, Neither one of us would recommend this place.]"
495,"[Went here on a weekday night., I didn't like it, and I probably won't be back.]"
729,[Tried to call in an order today & the phone was disconnected.]
1198,"[They have an entertainment book advertisement in the window but they won't accept the entertainment book coupon itself..., What gives?]"
1572,"[OMG!!!!, The Monte Cristo is out of this world!!!!, The ingredients do not sound like they fit together, but the flavor explodes in your mouth!!, If you have a chance to visit, do it!!!]"
...,...
14174,[Woman did not even turn around to hand me take out food and continued to talk to colleagues.]
14194,"[We were seated in a hibachi room, upon sitting down a mouse climbed up my sons chair., We shooed it away, then it would not leave., We informed the staff and left., Will never return.]"
14987,"[I hate to give this place a one star., We always eat here when we come to vegas., We did not know it was not Todd English anymore., The food became before we even got our water!!!, And the food sucked and so did the service., NEVER AGAIN!!, !]"
15145,"[The waiter kept wiping his nose with his hands and serving people their food., Without washing his hands., Ugh., Unfortunately I didn't catch his name., His hat was turned backwards.]"


In [0]:
import random
from collections import Counter

random.seed(RANDOM_SEED)

def get_top_n_pairs(group, n):
    counter = Counter()
    for pairs in group['noun-adjective']:
        for pair in pairs:
            counter[(pair[0].lower(), pair[1].lower())] += 1
    
    return counter.most_common(n)

for business_id, group in random.sample(list(YELP_REVIEWS.groupby('business_id')), 5):
    print(f"Top 10 noun-adjective pair for business '{business_id}':")
    top_pairs = get_top_n_pairs(group, 10)
    for i, (pair, count) in enumerate(top_pairs):
        print(f"{i + 1:>3}. {'-'.join(pair):<25} {count}")
    
    print("=" * 180)

Top 10 noun-adjective pair for business 'Gr_TkW3iFdgahixONGBsww':
  1. room-nice                 191
  2. hotel-great               145
  3. hotel-nice                141
  4. room-great                114
  5. room-clean                91
  6. room-good                 83
  7. hotel-front               75
  8. hotel-good                73
  9. room-front                68
 10. room-next                 68
Top 10 noun-adjective pair for business '2xrpo-LXV9uGIwpvy0dwUw':
  1. car-free                  128
  2. car-clean                 113
  3. groupon-free              98
  4. car-other                 81
  5. car-good                  80
  6. wash-free                 70
  7. car-first                 70
  8. car-bad                   67
  9. car-last                  63
 10. car-great                 56
Top 10 noun-adjective pair for business 'WO3L0pmtAO8ozspmaVdHIg':
  1. food-good                 109
  2. food-chinese              98
  3. oxtail-good               70
  4. soup-goo

##### Approach 3: using rule-based matching

In [0]:
# https://medium.com/@ashiqgiga07/rule-based-matching-with-spacy-295b76ca2b68
from spacy.matcher import Matcher
from tqdm import tqdm

#instantiate a new Matcher class object 
matcher = Matcher(SPACY.vocab)

# pattern for noun phrase
nounphrase1 = [{'DEP': 'nsubj', 'POS' : {'IN':['NOUN', 'PROPN' , 'PROP']}}]
nounphrase2 = nounphrase1 + [{ 'POS': 'CCONJ'}] + nounphrase1

# pattern for adjective phrase
adjphrase1 =  [{'POS': 'ADV', 'OP': '*'}, {'DEP': 'acomp','POS': 'ADJ'}]
adjphrase2 = [{'POS': 'ADV', 'OP': '*'}, {'POS': 'ADJ'}, {'POS': 'CCONJ'}, {'POS': 'ADV', 'OP': '*'}, {'POS': 'ADJ'}]

# add the pattern to matcher object
matcher.add("P1", None, [{'POS' : 'ADV', 'OP': '*'}, {'POS' : 'ADJ', 'DEP': 'amod'}, {'POS': 'NOUN'}])
matcher.add("P2", None, nounphrase1 + [{'POS':'VERB'}] + adjphrase1)
matcher.add("P3", None, nounphrase1 + [{'POS':'VERB'}] + adjphrase2)
matcher.add("P4", None, nounphrase1 + [{'OP' :'*' , 'LENGTH': {'>': 1}}] + adjphrase2)
matcher.add("compound subj P1", None, [{'POS' : 'ADV', 'OP': '*'}, {'POS' : 'ADJ', 'DEP': 'amod'}, {'POS': 'NOUN'}, { 'POS': 'CCONJ'}, {'POS': 'NOUN'}] )
matcher.add("compound subj P2", None, nounphrase2 + [{'POS':'VERB'}] + adjphrase1)
matcher.add("compound subj P3", None, nounphrase2 + [{'POS':'VERB'}] + adjphrase2)
matcher.add("compound subj P4", None, nounphrase2 + [{'OP' :'*' , 'LENGTH': {'>': 1}}] + adjphrase2)

YELP_REVIEWS['matches'] = ''
for index, row in tqdm(list(YELP_REVIEWS.iterrows())):
    matchlist = set()
    if row['segments'] != "":
        for segment in row['segments']:
            doc = SPACY(segment)
            # Call the matcher object the document object and it will return match_id, start and stop indexes of the matched words
            matches = matcher(doc)
            # Extract matched results
            matchlist.update([doc[start:end].text for _, start, end in matches])

        YELP_REVIEWS.at[index, 'matches'] = list(matchlist)

num_empty = (YELP_REVIEWS['matches'].str.len() == 0).sum()
print(f"\nNumber of reviews without matches: {num_empty} ({num_empty / len(YELP_REVIEWS) * 100}%)")
display(YELP_REVIEWS[['text', 'stars', 'matches']])

100%|██████████| 15222/15222 [23:52<00:00, 10.63it/s]

88835

Number of reviews without matches: 498 (3.2715806070161606%)


,text,stars,matches
0,"We had my Mother's Birthday Party here on 10/29/16. What a Great time we all had. The food, music and waiters were Great!!! Thanks Lyles!!!",5.0,[Great time]
1,"Good Korean grill near Eaton Centre. The marinate is good. We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly. The fish fillet was bland and liver was meh. Salmon and chicken was really flavourable. Such a fun place to eat at for a date or group of friends. Even alone. No judgments here. \nThe staff is attentive, nice and considerate. Bigger groups will most likely be seated on the second floor which is way bigger.\nCaution: will smell like BBQ grill after.",4.0,"[Korean grill, marinate is good, fun place, fillet was bland, second floor, Bigger groups, liver was meh, staff is attentive]"
2,"Was recommended to try this place by few people and today was my first time here. All I can say is, I am coming back very soon.\n\nSERVICE\nWasn't sure if the guy was the owner but he was friendly and talked story while we waited for our food. Loved it!! Food came out within 10 min. \n\nFOOD\nTried hamburger steak and it was so delicious. Gravy/sauce they put on the hamburger steak was perfect! Also came with onion rings on top which I love. Chicken katsu was amazing! Chicken katsu here is crunchy and surprisingly has a flavor by itself that you really don't need a sauce for it. Best chicken katsu I had. \n\nOVERALL\nIt was a journey to get to this place as it took about 30min from my house but the service and food here made it worth the drive. I also love how they had a poster of Keali'i Reichel. (They had other posters but Keali'i Reichel happens to be my favorite). Place is clean, service is fast and friendly and food is delicious. What more could you ask for?",5.0,"[Best chicken, few people and today, Place is clean, service is fast, food is delicious, first time, katsu was amazing, few people, service is fast and friendly, other posters]"
3,"Ambience: Would not expect something this nice at Cannery Hotel but it is the nicest looking restaurant there. More for couples than group gatherings.\n\nService: The ambience & food make up for this, which unfortunately for us, the service has been terrible. We have come fairly close to restaurant closing both times (within the hour), but they do close very early for Vegas. The staff makes it VERY clear that they want to go home right from the start in hurrying orders and are more aggressive as time goes on. Unfortunate.\n\nFood: Very good. A little salty on some items during our first visit but good overall and again, warrants the overall 3 stars. Steak. Scallops wrapped in bacon. Calamari. Cobb salad. etc.",3.0,[first visit]
4,"Absolutely the WORST pool company that I have EVER had to deal with. The customer service is horrible. After leaving many messages over the course of a few weeks I was only able to contact them when I called them AGAIN. I asked to speak with the actual pool tech who initially came to my house. The RUDE lady on the phone told me that she was more than capable to answer my questions - about a pump that SHE HAS NOT SEEN, and about a conversation I had with the tech THAT SHE DID NOT HEAR. \n\nI was assigned to them by my home warranty company, and I will be filing a serious complaint with them and the BBB. I was told to take the cash out option from the warranty company for the part and then they would do the work and I could just pay them directly. After I received the cash out and called to schedule the appointment I was told that I need to replace the entire pool pump system and that would cost an additional $400 and that there was an electrical problem and that it would cost...",1.0,"[service is horrible, additional money, actual pool, entire pool, serious complaint, few weeks, electrical problem, many messages, WORST pool]"
...,...,...,...
15217,This was the worst experience ever. So much so that I'm in the parking lot writing this. Girl took off my entire eyebrow basical

In [0]:
# display results without match
empty = YELP_REVIEWS[YELP_REVIEWS['matches'].str.len() == 0]
display(empty[['segments']])

,segments
5,"[I love love their Kalbi, I always order it, the sauce is what makes it really good, ..hmm think I wanna eat that today.i didn't like their noodles to sweet..]"
46,"[I had LASIK with Dr. Wellish in 2004., My vision was horrible- 20/2700., Eleven years later and still no glasses., My vision has held steady at 20/40, and I still don't need readers at age forty-five., LASIK was life changing., If you're considering it, do it., Dr Wellish is the best., The office was organized & on time., They know what they're doing.]"
52,"[My wife and I looked forward to eating here for weeks., We decided to get one of our favorites tonight, bbq pork ribs., Let me say... extremely disapointed!, Neither one of us would recommend this place.]"
57,"[This is the worst Denny's ever , I mean nobody expects much from Denny's let's be honest , so I call to order no answer 20 rings , call back get put on hold then I say I'll start driving there to pick up the salmon , ok nobody comes back on phone , I walk in no line , nothing awful !!!]"
167,"[Keep trying to unsubscribe from their mailing list, though today they sent me 7 SPAM emails., Stop it!, They also have a link to review their spa's on Yelp on the e-mail - though it just goes to the Yelp homepage., They obviously don't care much about their customers., It was so long ago, I was there, I can't remember the service.]"
...,...
15145,"[The waiter kept wiping his nose with his hands and serving people their food., Without washing his hands., Ugh., Unfortunately I didn't catch his name., His hat was turned backwards.]"
15154,"[Go see Marieja (so sorry if I spelled that wrong)!, She is fantastic., My mom & I both visited her for spur-of-the-moment eyebrow waxes and tints and our eyebrows have seriously never looked better., She is totally professional and really knows what she's doing., So happy I found her., I highly recommend her, and I'm very picky about who does my eyebrows.]"
15195,"[Auto Tint Express was recommended to me., There are so many out there, so you never know what you are going to get., Well they are great., Quality of work and customer service is excellent., They know their stuff and guarantee their work and prices are affordable., Thank you Mario and team.]"
15210,[It's definitely better than Kyoto AYCE :) and cheaper]


In [0]:
class PhraseMatcher:
    def __init__(self, patterns):
        self.matcher = Matcher(SPACY.vocab)

        for name, pattern in patterns.items():
            self.matcher.add(name, None, pattern)

    def get_longest_match(self, doc):
        longest_match = None
        for _, start, end in self.matcher(doc):
            span = doc[start:end] 
            if longest_match is None or len(span.text) > len(longest_match):
                longest_match = span.text

        return longest_match

# Matcher for noun phrase
nounphrase1 = [{'POS': {'IN':['NOUN', 'PROPN' , 'PROP']}}]
nounphrase2 = nounphrase1 + [{ 'POS': 'CCONJ'}] + nounphrase1
nounphrase_matcher = PhraseMatcher({"NP1": nounphrase1, "NP2": nounphrase2})

# Matcher for adjective phrase
adjphrase1 =  [{'POS': 'ADV', 'OP': '*'}, {'POS' : 'ADJ'}]
adjphrase2 = [{'POS': 'ADV', 'OP': '*'}, {'POS' : 'ADJ'}, {'POS': 'CCONJ'}, {'POS' : 'ADV', 'OP': '*'}, {'POS' : 'ADJ'}]
adjphrase_matcher = PhraseMatcher({"ADJP1": adjphrase1, "ADJP2": adjphrase2})

YELP_REVIEWS['noun-adjective'] = ''
for index, row in tqdm(list(YELP_REVIEWS.iterrows())):
    noun_adjectives = set()
    if row['matches'] != "":
        for x in row['matches']:
            doc = SPACY(x)
            # Extract noun phrase
            noun = nounphrase_matcher.get_longest_match(doc)
            # Extract adjective phrases
            adjective = adjphrase_matcher.get_longest_match(doc)

            if noun is not None and adjective is not None:
                noun_adjectives.add((noun, adjective))

    YELP_REVIEWS.at[index, 'noun-adjective'] = list(noun_adjectives)

display(YELP_REVIEWS[['matches', 'stars', 'noun-adjective']])

100%|██████████| 15222/15222 [12:37<00:00, 20.10it/s]


,matches,stars,noun-adjective
0,[Great time],5.0,"[(time, Great)]"
1,"[Korean grill, marinate is good, fun place, fillet was bland, second floor, Bigger groups, liver was meh, staff is attentive]",4.0,"[(marinate, good), (fillet, bland), (staff, attentive), (liver, meh), (floor, second), (grill, Korean), (groups, Bigger)]"
2,"[Best chicken, few people and today, Place is clean, service is fast, food is delicious, first time, katsu was amazing, few people, service is fast and friendly, other posters]",5.0,"[(Place, clean), (katsu, amazing), (people, few), (time, first), (people and today, few), (service, fast), (posters, other), (food, delicious), (service, fast and friendly), (chicken, Best)]"
3,[first visit],3.0,"[(visit, first)]"
4,"[service is horrible, additional money, actual pool, entire pool, serious complaint, few weeks, electrical problem, many messages, WORST pool]",1.0,"[(service, horrible), (money, additional), (pool, WORST), (messages, many), (pool, entire), (weeks, few), (problem, electrical), (pool, actual), (complaint, serious)]"
...,...,...,...
15217,"[entire eyebrow, worst experience]",1.0,"[(eyebrow, entire), (experience, worst)]"
15218,[food is great],5.0,"[(food, great)]"
15219,"[French bread, food is not very good, new owners, last week, other people, time was disappointing, cranberry toast, French bread or something, other bread, roasted chicken, now half way, great food, iced tea, Several minutes, particular time, other options, people are thrilled, half way, several minutes]",1.0,"[(owners, new), (way, now half), (bread or something, French), (minutes, Several), (food, great), (options, other), (chicken, roasted), (people, other), (week, last), (bread, other), (food, not very good), (time, disappointing), (bread, French), (minutes, several), (toast, cranberry), (time, particular)]"
15220,[food was delicious],5.0,"[(food, delicious)]"


In [0]:
import random
from collections import Counter

random.seed(RANDOM_SEED)

def get_top_n_pairs(group, n):
    counter = Counter()
    for pairs in group['noun-adjective']:
        for pair in pairs:
            counter[(pair[0].lower(), pair[1].lower())] += 1

    return counter.most_common(n)

for business_id, group in random.sample(list(YELP_REVIEWS.groupby('business_id')), 10):
    print(f"Top 10 noun-adjective pair for business '{business_id}':")
    top_pairs = get_top_n_pairs(group, 10)
    for i, (pair, count) in enumerate(top_pairs):
        print(f"{i + 1:>3}. {'-'.join(pair):<25} {count}")
    
    print("=" * 180)

Top 10 noun-adjective pair for business 'Gr_TkW3iFdgahixONGBsww':
  1. desk-front                16
  2. hotel-great               7
  3. hotel-best                5
  4. bar-mini                  4
  5. room-clean                4
  6. staff-friendly            4
  7. food-good                 4
  8. hotel-beautiful           4
  9. time-great                3
 10. door-next                 3
Top 10 noun-adjective pair for business '2xrpo-LXV9uGIwpvy0dwUw':
  1. job-great                 8
  2. time-first                7
  3. job-good                  6
  4. staff-friendly            6
  5. locations-other           5
  6. car-clean                 5
  7. car-best                  4
  8. time-next                 4
  9. prices-great              4
 10. car-classic               4
Top 10 noun-adjective pair for business 'WO3L0pmtAO8ozspmaVdHIg':
  1. food-chinese              19
  2. food-fast                 13
  3. night-late                10
  4. dog-hot                   8
  5. d